# 주가 데이터 크롤링

In [55]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from urllib import parse

In [144]:
# 주가 크롤링
company = '삼성바이오로직스'
code = '207940'
start = datetime.datetime(2020, 1, 20)
end = datetime.datetime(2021, 9, 15)
df_null = fdr.DataReader(code, start, end)     # 005930 : 삼성전자
df_null.loc[df_null['Volume']==0]
df_null['Volume'] = df_null['Volume'].replace(0, np.nan)
df = df_null.dropna()                              # 결측치 제거

In [145]:
# 새로운 칼럼 생성
# (Price : 당일 대비 다음날 주가가 상승했으면 1, 하락했으면 0 표시)
df['Price'] = 0
for i in range(df.shape[0]-1):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

# 파일 저장
df.to_csv('Analysis/{}_주가데이터.csv'.format(company))

c:\users\jysi\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\jysi\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [146]:
# 수집날짜 리스트 생성
price_data = pd.read_csv('Analysis/{}_주가데이터.csv'.format(company))
df_0 = price_data[price_data['Price']==0]['Date']
date_0 = []
for i in range(0, len(df_0)):
    date_0.append(str(df_0.tolist()[i]))

df_1 = price_data[price_data['Price']==1]['Date']
date_1 = []
for i in range(0, len(df_1)):
    date_1.append(str(df_1.tolist()[i]))

In [147]:
len(date_0),len(date_1)

(218, 195)

In [148]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

In [149]:
text = company
text = parse.urlencode({'query':text}, encoding='euc-kr')
name = text.replace('uery', '')

In [150]:
def naver_news_view(name, dates):
    base_url = 'https://finance.naver.com/news/news_search.nhn?rcdate=&{}&x=0&y=0&sm=all.basic&pd=4&stDateStart={}&stDateEnd={}'
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    }
    
    for date in dates:
        url = base_url.format(name, date, date)
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text)
            title_list = soup.select('dd.articleSubject')
            for title in title_list:
                try:
                    news_title = title.text.strip()
                    result_list.append([news_title])
                except:
                    error_cnt += 1
            title_list2 = soup.select('dt.articleSubject')
            for title in title_list2:
                try:
                    news_title = title.text.strip()
                    result_list.append([news_title])
                except:
                    error_cnt += 1

In [151]:
result_list = []
error_cnt = 0

naver_news_view(name, date_0)
title_df_0 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_0['주가변동'] = 0

In [11]:
title_df_0

,뉴스제목,주가변동
0,"23일, 외국인 거래소에서 KODEX 200TR(-1.38%), 삼성바이오로직스(...",0
1,"'엔씨소프트' 52주 신고가 경신, L2M 흥행, 매출처 다변화 시작 - KB증권...",0
2,'실적' 있는 곳에 '주가' 따라 온다,0
3,"[빅데이터MSI]시장심리 톱5, 현대차·기아차·엔씨소프트·현대위아·락앤락",0
4,"[Hot-Line] ""엔씨소프트, 올해 실적·신작 출시 모멘텀 예정""",0
...,...,...
3281,"고용부 장관, 네이버 등 IT기업에 ""하반기 채용확대해달라""",0
3282,"네이버·넥슨 등 IT기업 수장 만난 고용장관 ""채용문 열어달라""",0
3283,"엔씨 블소2, 8월 모바일 매출 29위 안착",0
3284,엔씨가 달라졌다?…'블소2' 업데이트에 3주 연속 유저 목소리 반영,0


In [152]:
result_list = []
error_cnt = 0

naver_news_view(name, date_1)
title_df_1 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_1['주가변동'] = 1

In [13]:
title_df_1

,뉴스제목,주가변동
0,10년간 시총 수십배…코스피 '슈퍼 스타'들,1
1,'三電만 올랐냐? 나도 올랐다'…삼전 상승률 압도한 우량주는?,1
2,1분기 상장사 실적 청신호…게임 뜨고 항공 진다,1
3,"21일, 거래소 외국인 순매도상위에 화학 업종 6종목",1
4,"블록체인 게임 개발사 수퍼트리, 30억 투자 유치",1
...,...,...
2566,"엔씨의 날개 없는 추락, 60만원대 깨졌다",1
2567,'날개 없는 추락' 엔씨 60만원 붕괴,1
2568,추석 연휴를 앞두고 관망세‥코스닥 1030선 이탈,1
2569,엔씨소프트 끝없는 추락..주당 60만원대도 붕괴,1


In [153]:
import re

def preprocessor(text):
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", " ", text)
    return text

In [154]:
title_df = pd.concat([title_df_0, title_df_1])
title_df['뉴스제목'] = title_df['뉴스제목'].apply(preprocessor)
title_df.to_csv('Analysis/{}_뉴스타이틀.csv'.format(company), index=False, encoding='utf-8')

# Test Code

In [106]:
result_list = []
base_url = "https://finance.naver.com/news/news_search.nhn?rcdate=&{}&x=0&y=0&sm=all.basic&pd=4&stDateStart={}&stDateEnd={}"
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
}

text = '삼성전자'
text = parse.urlencode({'query':text}, encoding='euc-kr')
text = text.replace('uery', '')
date = '2021-08-31'

url = base_url.format(text, date, date)
res = requests.get(url, headers=headers)
if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    title_list = soup.select('dd.articleSubject')
    for title in title_list:
        try:
            news_title = title.text.strip()
            result_list.append([news_title])
        except:
            error_cnt += 1
test = pd.DataFrame(result_list)
test

,0
0,삼성전자 'Z플립·폴더3 돌풍에…부품주도 날았다
1,"""떨어질 만큼 떨어졌나""…올해 삼성전자 20조 판 외국인 하루 3600억 사들였다"
2,"삼성전자, 2Q 파운드리 점유율 17.3%...1위 TSMC 52.9%"
3,"추미애, 언론법 9월 처리에 쓴소리 ""내일로 미룬다고 박수 안쳐"""
4,"“폴더블폰, 아직은 너무 두꺼워요” 더 얇아질 수는 없을까"
5,이건희 컬렉션·軍부대 비데 설치·유기동물 관리비… 눈길 끄는 이색사업[2022 예...
6,김총리의 지지…이재용 경영복귀 힘실리나
7,"하반기 메모리값 ""겨울 온다"" vs ""수요 견고"" 팽팽"
8,‘폴더블폰 인기 수혜’ KH바텍 또 신고가
9,[영상] 고문(?) 받는 '갤Z폴드3'…극한 테스트에서 '내구성' 입증


In [5]:
text_df = pd.read_csv('Analysis/005930_뉴스타이틀.csv')
text_df

,뉴스제목,주가변동
0,"정부 ""삼성·LG 세탁기, 美 세이프가드서 제외해야""",0
1,"정부, 美 세탁기 세이프가드 공청회서 반대 입장 피력",0
2,"이총리 ""최저임금 인상·노동시간 단축 정책 연착륙 노력""",0
3,"박용만 ""끊임없는 혁신이 미래 성장 담보하는 유일한 길""",0
4,"이재용 삼성전자 부회장, 보아오 포럼 이사 물러날듯",0
...,...,...
18015,아마존·테슬라 '엔비디아·ARM 결합' 반대,1
18016,"[단독] 김부겸 총리, 다음 주 ‘KT·삼바’ 방문…이재용 부회장 만나나",1
18017,삼성 美 신규 파운드리 ‘텍사스·뉴욕’ 압축,1
18018,한국 vs 미국 반도체 공장 입지 비교,1


In [6]:
import re
from time import time

def preprocessor(text):
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", " ", text)
    return text

text_df['뉴스제목'] = text_df['뉴스제목'].apply(preprocessor)
text_df

전처리 시작
전처리 완료: 소요시간 [0]초


,뉴스제목,주가변동
0,정부 삼성 LG 세탁기 세이프가드서 제외해야,0
1,정부 세탁기 세이프가드 공청회서 반대 입장 피력,0
2,이총리 최저임금 인상 노동시간 단축 정책 연착륙 노력,0
3,박용만 끊임없는 혁신이 미래 성장 담보하는 유일한 길,0
4,이재용 삼성전자 부회장 보아오 포럼 이사 물러날듯,0
...,...,...
18015,아마존 테슬라 엔비디아 ARM 결합 반대,1
18016,단독 김부겸 총리 다음 주 KT 삼바 방문 이재용 부회장 만나나,1
18017,삼성 신규 파운드리 텍사스 뉴욕 압축,1
18018,한국 vs 미국 반도체 공장 입지 비교,1
